In [ ]:
import os
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

In [ ]:
# Конфігураційні константи
DATA_DIR = Path("../Dataset")
OUTPUT_DIR = Path("../Filtered datasets")

# мапування днів неділі в timestamp
DATE_MAP: Dict[str, str] = {
    'Monday': '2023-11-06 12:00:00',
    'Tuesday': '2023-11-07 12:00:00',
    'Wednesday': '2023-11-08 12:00:00',
    'Thursday-Morning': '2023-11-09 09:00:00',
    'Thursday-Afternoon': '2023-11-09 15:00:00',
    'Friday-Morning': '2023-11-10 09:00:00',
    'Friday-Afternoon1': '2023-11-10 13:00:00',
    'Friday-Afternoon2': '2023-11-10 17:00:00',
}

In [ ]:
CATEGORY_LABELS: Dict[str, List[str]] = {
    'BENIGN': ['BENIGN'],
    'DoS': ['DDoS', 'DoS slowloris', 'DoS Hulk', 'DoS GoldenEye'],
    'PortScan': ['PortScan'],
    'Bot_Infiltration': ['Bot', 'Infiltration'],
    'Web': ['Web Attack – Brute Force', 'Web Attack – XSS', 'Web Attack – Sql Injection'],
    'FTP_SSH_Patator': ['FTP-Patator', 'SSH-Patator'],
    'Heartbleed': ['Heartbleed'],
}

GROUP_FEATURES: Dict[str, List[str]] = {
    'dos': ['Fwd Packets/s', 'Bwd Packets/s', 'Flow Duration', 'Flow IAT Min', 'Flow IAT Max', 'SYN Flag Count', 'PSH Flag Count'],
    'portscan': ['SYN Flag Count', 'FIN Flag Count', 'RST Flag Count', 'Total Fwd Packets', 'Total Backward Packets'],
    'bot_infiltration': ['Flow Duration', 'Fwd IAT Std', 'Bwd IAT Std', 'Fwd PSH Flags', 'Bwd URG Flags', 'Down/Up Ratio'],
    'web': ['Fwd Header Length', 'Bwd Header Length', 'Packet Length Variance', 'ACK Flag Count', 'Average Packet Size'],
    'ftp_ssh_patator': ['Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Bwd Avg Bytes/Bulk', 'Active Mean', 'Idle Mean', 'Init_Win_bytes_forward'],
    'heartbleed': ['Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd IAT Min', 'Total Length of Fwd Packets', 'Packet Length Std'],
}

BASE_FEATURES: List[str] = [
    'Flow Bytes/s', 'Flow Packets/s', 'Average Packet Size', 'Down/Up Ratio',
    'Packet Length Mean', 'Packet Length Std', 'Min Packet Length', 'Max Packet Length',
    'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 'Bwd IAT Mean',
    'SYN Flag Count', 'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
    'Active Mean', 'Idle Mean', 'Subflow Fwd Packets', 'Subflow Bwd Packets',
    'Label', 'dow', 'hour', 'dow_sin', 'dow_cos', 'hour_sin', 'hour_cos'
]

STD_FEATURES: List[str] = [
    'Fwd Packet Length Std', 'Bwd Packet Length Std', 'Flow IAT Std',
    'Fwd IAT Std', 'Bwd IAT Std', 'Packet Length Std', 'Active Std', 'Idle Std'
]

In [ ]:
def load_and_concat_csvs(data_dir: Path) -> pd.DataFrame:
    """
    Завантажує всі CSV-файли з директорії, додає стовпець 'Day' на основі імені файлу
    та об'єднує їх в один DataFrame.

    :param data_dir: шлях до директорії з CSV-файлами
    :return: конкатенований DataFrame з сирими даними
    """
    csv_paths = sorted(data_dir.glob("*.csv"))
    dfs: List[pd.DataFrame] = []

    for path in csv_paths:
        day_label = path.stem  # мітка дня з імені файлу
        df_temp = pd.read_csv(path)
        df_temp['Day'] = day_label
        dfs.append(df_temp)

    concatenated = pd.concat(dfs, ignore_index=True)
    concatenated.columns = concatenated.columns.str.strip()  # очищення пробілів в назвах стовпців
    return concatenated

In [ ]:
def add_datetime_index(df: pd.DataFrame, date_map: Dict[str, str]) -> pd.DataFrame:
    """
    Перетворює стовпець 'Day' у datetime-індекс на основі мапи дат,
    встановлює його як індекс і видаляє стовпець 'Day'.

    :param df: DataFrame з колонкою 'Day'
    :param date_map: словник мітка -> datetime рядок
    :return: DataFrame з datetime-індексом
    """
    df['timestamp'] = pd.to_datetime(df['Day'].map(date_map))
    df = df.set_index('timestamp').drop(columns=['Day'])
    return df

In [ ]:
def engineer_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Додає циклічні ознаки для дня тижня та години (синус/косинус).

    :param df: DataFrame з datetime-індексом
    :return: DataFrame з новими часовими ознаками
    """
    df['dow'] = df.index.dayofweek  # день тижня (0=Понеділок)
    df['hour'] = df.index.hour  # година доби
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df

In [ ]:
def save_grouped_by_category(df: pd.DataFrame, output_dir: Path) -> None:
    """
    Для кожної категорії у CATEGORY_LABELS вибирає базові та додаткові ознаки,
    а потім зберігає результуючі CSV у вказаній директорії.

    :param df: очищений DataFrame з колонкою 'Label'
    :param output_dir: директорія для збереження CSV
    """
    output_dir.mkdir(parents=True, exist_ok=True)

    for category, labels in CATEGORY_LABELS.items():
        if category == 'BENIGN':
            features = BASE_FEATURES
        else:
            key = category.lower()
            extra = GROUP_FEATURES.get(key, [])
            features = list(dict.fromkeys(BASE_FEATURES + extra))

        subset = df[df['Label'].isin(labels)][features]
        filepath = output_dir / f"{category}.csv"
        subset.to_csv(filepath, index=False)
        print(f"Збережено {len(subset)} рядків до {filepath}")

In [ ]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Очищення даних:
      - Негативні числа у числових колонках замінюються на NaN
      - Нескінченності замінюються на NaN
      - Видалення рядків з будь-якими NaN
      - Скидання індексу

    :param df: початковий або частково оброблений DataFrame
    :return: очищений DataFrame, готовий до аналізу або моделювання
    """
    # Визначення числових колонок (без деяких виключень)
    exclude = ['Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Label']
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(exclude)

    # Маска негативних значень та нескінченностей
    df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Видалення рядків з пропусками
    df = df.dropna(axis=0, how='any')

    # Скидання індексу
    df = df.reset_index(drop=True)
    return df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,dow_sin,dow_cos,hour_sin,hour_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-10 13:00:00,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,0,0.0,0.0,0,0,BENIGN,-0.433884,-0.900969,-0.258819,-0.965926
2023-11-10 13:00:00,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,0,0.0,0.0,0,0,BENIGN,-0.433884,-0.900969,-0.258819,-0.965926
2023-11-10 13:00:00,22,160,1,1,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,BENIGN,-0.433884,-0.900969,-0.258819,-0.965926
2023-11-10 13:00:00,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,0,0.0,0.0,0,0,BENIGN,-0.433884,-0.900969,-0.258819,-0.965926
2023-11-10 13:00:00,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,BENIGN,-0.433884,-0.900969,-0.258819,-0.965926


In [ ]:
raw_df = load_and_concat_csvs(DATA_DIR)
df_indexed = add_datetime_index(raw_df, DATE_MAP)
df_features = engineer_time_features(df_indexed)
df_clean = clean_data(df_features)
save_grouped_by_category(df_clean, OUTPUT_DIR)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co